<a href="https://colab.research.google.com/github/assuncaomarcos/classroom_extensions/blob/main/notebooks/Server_Side_JavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Node.js Extension
--------------------

This notebook demonstrates how to use the Node extension to run JavaScript on the node hosting the notebook and the customizations to the `%%javascript` cell magic.

## Installing the Classroom Extensions
------------------

We will install the extensions via pip+github:

In [ ]:
!pip3 install classroom-extensions

Once the extensions are installed, you can load the `node` extension to customize the `%%javascript` cell magic:

In [2]:
%load_ext classroom_extensions.node

## Server-Side JavaScript

The extension adds a few parameters to the `%%javascript` magic:

* `-t | --target`: specifies where the script must run or be stored. The possible choices are `browser`, `node`, and `disk`. The default value is `browser`, which is the default behavior of letting the browser execute the code pushed to the result section of the cell. When specifying `node`, the cell contents will be saved to a file on the Jupyter server running the notebook and executed using Node.js. The option `disk` is a helper that stores the cell contents into a file. It is similar to `%%writefile` but allows for syntax highlight when running on Colab.

* `-f | --filename`: specifies the filename to save the cell contents for execution.

* `-p |--port`: the port number in case the JavaScript code will launch a server listing to a given port. The value of this parameter gets exported to a `NODE_PORT` environment variable that the JavaScript code can use.

The following provides a few examples:

In [3]:
%%javascript --target=node --filename=example1.js

const fs = require('fs');

const filePath = '/etc/lsb-release';

fs.readFile(filePath, 'utf8', (err, data) => {
  if (err) {
    console.error(`Error reading file: ${err}`);
    return;
  }

  console.log('File contents:');
  console.log(data);
});

File contents:
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.2 LTS"



In [4]:
%%javascript --target=node --filename=http_server.js --port=3000

const http = require('http');

const hostname = 'localhost';
const port = process.env.NODE_PORT || 3000;

const server = http.createServer((req, res) => {
    res.statusCode = 200
    res.setHeader('Content-Type', 'text/plain')
    res.end('Hello World!\n')
});

server.listen(port, hostname, () => {
    console.log(`Server listening at https://${hostname}:${port}/`)
});

Server listening at https://localhost:3000/


The extension will automatically kill any node process previously started to listen on the provided port. When running on Colab, you can expose the proxy address that makes the server accessible from outside (on Chrome):

In [5]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(3000)"))

https://hye6of32s8c-496ff2e9c6d22116-3000-colab.googleusercontent.com/


To simply save a given file while benefiting from syntax highlighting on Colab:

In [6]:
%%javascript --target=disk --filename=formatdate.js

const ordinal = require("ordinal");
const {days, months} = require("date-names");

exports.formatDate = function(date, format) {
    return format.replace(/YYYY|M(MMM)?|Do?|dddd/g, tag => {
        if (tag == "YYYY") return date.getFullYear();
        if (tag == "M") return date.getMonth();
        if (tag == "MMMM") return months[date.getMonth()];
        if (tag == "D") return date.getDate();
        if (tag == "Do") return ordinal(date.getDate());
        if (tag == "dddd") return days[date.getDay()];
    });
};

In [ ]:
!npm install ordinal
!npm install date-names

In [8]:
%%javascript --target=node --filename=use_formatdate.js

const {formatDate} = require("./formatdate");
console.log(formatDate(new Date(2023, 5, 10), "MMMM, dddd the Do"));

June, Saturday the 10th


## Browser-Side JavaScript with Console

The extension also customizes the `%%javascript` magic so that the result section of the cell mimics the browser's console. The results to calls to the main console methods will display their results in the output of the cell:

In [9]:
%%javascript

console.log("This is a log message");
console.warn("This is a warning message");
console.error("This is an error message");

console.log("This is another log message");
console.error("This is another error message");

<classroom_extensions.node.JavascriptWithConsole object>

Here a more involved example:

In [10]:
%%javascript

function fetchJSON(url) {
  return fetch(url).then(response => {
    if(!response.ok) {
      throw new Error(`HTTP Error: ${response.status}`);
    } else {
      return response.json();
    }
  })
  .catch(e => {
    console.log(`Error retrieving ${url}: ${e.message}`);
  });
}

// This example used the Nobel Prizes API
const categories = ["che", "med", "lit", "pea", "eco", "phy"];
const baseUrl = 'https://api.nobelprize.org/2.1/nobelPrizes?nobelPrizeYear=2021&nobelPrizeCategory=';

// A list of promises to fetch the prizes for each category
const promises = [];
categories.forEach(category => {
  promises.push(fetchJSON(baseUrl + category));
});

// Wait until all promises are fulfilled
Promise.all(promises).then(values => {
  values.forEach(json => {
    category = json.nobelPrizes[0].category.en;
    firstLaureate = json.nobelPrizes[0].laureates[0].fullName.en;
    console.log(`${category}: ${firstLaureate}`);
  });
});

<classroom_extensions.node.JavascriptWithConsole object>